<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 35px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Nuclear Incidents
  </div> 

  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 25px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
      Topic modeling - Title-level
  </div> 


  <div style=" float:left; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jean-baptiste AUJOGUE
  </div> 
  
  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Jan 2023
  </div> 

<a id="TOC"></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import warnings
import os

# data 
import numpy as np
import pandas as pd

# viz
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")
print('python version :', sys.version)

**Path to data repertory**

In [ ]:
path_to_repo = os.path.dirname(os.getcwd())
path_to_data = os.path.join(path_to_repo, 'data')
path_to_save = os.path.join(path_to_repo, 'saves')

In [ ]:
path_to_repo

In [ ]:
sys.path.insert(0, os.path.join(path_to_repo, 'src'))

In [ ]:
from tmtools.tfidf import compute_stripped_ngrams_fr, compute_sklearn_tfidf_matrix
from tmtools.topic import compute_topic_modeling, plot_topic_words

<a id="classification"></a>

# 1. Tfidf feature matrix

[Table of Content](#TOC)

In [ ]:
df_corpus = pd.read_excel(os.path.join(path_to_data, 'data.xlsx'))
df_corpus.labels.fillna(value = '', inplace = True)

In [ ]:
df_corpus.head(5)

In [ ]:
corpus = df_corpus.title.tolist()
len(corpus)

#### Compute stripped ngrams

In [ ]:
vocab = compute_stripped_ngrams_fr(
    corpus, 
    sublinear_tf = True,
    use_idf = True,
    ngram_range = (1, 3),
    min_df = 1,
    strip_accents = None,
    lowercase = True,
)

In [ ]:
len(vocab)

In [ ]:
tfidf_matrix, tfidf_ngrams = compute_sklearn_tfidf_matrix(
    corpus, 
    vocab,
    sublinear_tf = True,
    use_idf = True,
    ngram_range = (1, 3),
    min_df = 3,
    strip_accents = None,
    lowercase = True,
)
tfidf_matrix.shape

# 2. Topic modeling

[Table of Content](#TOC)

In [ ]:
def retain_relevant_topics(df_text_topic, sim_threshold, sim_tolerance):
    max_sims = df_text_topic.max(axis = 1)
    df_tmp = df_text_topic[max_sims >= sim_threshold]
    df_tmp = df_tmp.apply(
        func = lambda r: tuple(i+1 for i, v in enumerate(r) if v >= sim_tolerance * max_sims[r.name]),
        axis = 1,
    )
    return df_tmp.to_dict()

def retain_interpretated_topics(relevant_topics_dict, topic_interpretation_dict):
    interpretated_topics_dict = {k: tuple(j for j in v if j in topic_interpretation_dict) for k, v in relevant_topics_dict.items()}
    interpretated_topics_dict = {k: tuple(topic_interpretation_dict[j] for j in v) for k, v in interpretated_topics_dict.items()}
    interpretated_topics_dict = {k: v for k, v in interpretated_topics_dict.items() if v}
    return interpretated_topics_dict

In [ ]:
n_topics = 100

#### Topic modeling using LSA

In [ ]:
df_text_topic_LSA, df_topic_feature_LSA, df_topic_importance_LSA, df_feature_importance_LSA = compute_topic_modeling(
    tfidf_matrix, tfidf_ngrams, method = 'LSA', n_components = n_topics, topic_name = 'Title topic',
)

In [ ]:
df_text_topic_LSA.head()

In [ ]:
LSA_relevant_topics = retain_relevant_topics(df_text_topic_LSA, sim_threshold = 0.2, sim_tolerance = 0.8)
LSA_relevant_topics

#### Interpretation of LSA topics

In [ ]:
# LSA
# TODO: only interpret topics present in dict of relevant topis
plot_topic_words(df_topic_feature_LSA, n_topics = 100, n_top_words = 15)

In [ ]:
df_topic_importance_LSA.plot(figsize = (15, 5))

In [ ]:
LSA_relevant_topics_flat = pd.Series([j for v in LSA_relevant_topics.values() for j in v])
LSA_relevant_topics_flat.value_counts().plot.barh(figsize = (10, 15)).invert_yaxis()
plt.show()

In [ ]:
LSA_topic_interpretation = {
    1: "Non-respect des spécifications techniques d'exploitation",
    2: "Non-respect des règles générales d'exploitation",
    3: "Détection tardive d'indisponibilité",
    4: "Défaut d’isolement de l’enceinte de confinement",
    5: "Sortie du domaine de fonctionnement",
    6: "Incident de contamination",
    9: "Défaut du groupe électrogène de secours",
    10: "Défaut de maîtrise de la criticité",
    12: "Défaut de tenue au séisme",
    14: "Défaut du circuit d’injection de sécurité",
    16: "Défaut du turbo-alternateur de secours",
    17: "Non-respect d’une mesure compensatoire",
    18: "Défaut du circuit de refroidissement",
    20: "Défaut du circuit d’eau brute",
    21: "Défaut du groupe électrogène de secours",
    22: "Défaut du circuit de refroidissement",
    23: "Défaut de grappes de commande",
    24: "contamination d’outillages",
    28: "Rejets de fluide frigorigène",
    31: "Défaut de réalisation du contrôle périodique",
    34: "Surplus de matière uranifère",
    38: "Incident de contamination",
    39: "Défaut d’entreposage de déchets radioactifs",
    43: "Défaut de montée en puissance",
    52: "Défaut de détecteurs incendie",
    58: "Défaut du circuit de contrôle volumétrique et chimique",
}

In [ ]:
# raise warning if newly created label already exists in previously created labels
old_labels = {l for ls in df_corpus.labels for l in ls.split(';')} - {''}
new_labels = {j for v in LSA_interpretated_topics.values() for j in v}
intersect = new_labels & old_labels
if intersect:
    print('WARNING: The following list of created labels already exist in data:\n\t- ' + '\n\t- '.join(sorted(intersect)))

In [ ]:
LSA_interpretated_topics = retain_interpretated_topics(LSA_relevant_topics, LSA_topic_interpretation)
LSA_interpretated_topics

In [ ]:
LSA_interpretated_topics_flat = pd.Series([j for v in LSA_interpretated_topics.values() for j in v])
LSA_interpretated_topics_flat.value_counts().plot.barh(figsize = (10, 15)).invert_yaxis()
plt.show()

In [ ]:
# append labels to existing ones

In [ ]:
df_corpus['labels'] = pd.Series(df_corpus.index).apply(
    lambda i: ';'.join(((df_corpus.at[i, 'labels'],) if df_corpus.at[i, 'labels'] else ()) + (() if i not in LSA_interpretated_topics else LSA_interpretated_topics[i]))
)

#### Topic modeling using NMF

In [ ]:
df_text_topic_NMF, df_topic_feature_NMF, df_topic_importance_NMF, df_feature_importance_NMF = compute_topic_modeling(
    tfidf_matrix, tfidf_ngrams, method = 'NMF', n_components = n_topics, topic_name = 'Title topic',
)

In [ ]:
NMF_relevant_topics = retain_relevant_topics(df_text_topic_NMF, sim_threshold = 0.1, sim_tolerance = 0.8)
NMF_relevant_topics

#### Interpretation of NMF topics

In [ ]:
# LSA
plot_topic_words(df_topic_feature_NMF, n_topics = 100, n_top_words = 15)

In [ ]:
topic_NMF.value_counts().plot.barh(figsize = (10, 15)).invert_yaxis()
plt.show()

In [ ]:
NMF_topic_interpretation = {
    1: "Non-respect des spécifications techniques d'exploitation",
    2: "Non-respect des règles générales d'exploitation",
    3: "Détection tardive d'indisponibilité",
    4: "Défaut d’isolement de l’enceinte de confinement",
    5: "Sortie du domaine de fonctionnement",
    6: "Incident de contamination radioactive",
    7: "circuit d’alimentation de secours des générateurs de vapeur",
    8: "Défaut de maîtrise de la criticité",
    9: "Défaut du groupe électrogène de secours",
    10: "Défaut de maîtrise de la criticité",
    12: "Défaut de tenue au séisme",
    13: "Défaut du système de ventilation",
    14: "Défaut du circuit d’injection de sécurité",
    16: "Défaut du turbo-alternateur de secours",
    17: "Non-respect d’une mesure compensatoire",
    18: "Défaut du circuit de refroidissement",
    19: "Défaut de tenue au séisme",
    20: "Défaut du circuit d’eau brute",
    21: "Défaut du groupe électrogène de secours",
    22: "Défaut du circuit de refroidissement",
    23: "Défaut de grappes de commande",
    24: "Sortie du domaine de fonctionnement",
    26: "Contamination par effluents radioactifs",
    28: "Rejets de fluide frigorigène",
    30: "Défaut de réalisation du contrôle périodique",
    31: "Défaut de réalisation du contrôle périodique",
    32: "Défaut d’alimentation électrique",
    34: "Surplus de matière uranifère",
    36: "Défaut de capteur",
    37: "Incident lors du redémarrage du réacteur",
    38: "Incident de contamination",
    39: "Défaut d’entreposage de déchets radioactifs",
    41: "Arret automatique du réacteur",
    45: "Défaut de montée en puissance",
    46: "Incident dans la piscine d’entreposage du combustible",
    51: "Dépassement de délai de réparation",
    52: "Non-respect des règles générales d'exploitation",
    54: "Défaut des systèmes de protection du réacteur",
    57: "Défaut du circuit secondaire",
    61: "du circuit de contrôle volumétrique et chimique",
    63: "Sortie du domaine de fonctionnement",
    71: "Défaut de tenue au séisme",
    72: "Non-respect des règles générales d'exploitation",
    77: "Défaut d’intégrité de barrière de confinement",
    78: "Défaut sur un générateur de vapeur",
    79: "Non-respect des règles générales d'exploitation",
    80: "Défaut de système de filtration d’iode",
    87: "Défaut de clapets coupe-feu",
    88: "Non-respect des règles générales d'exploitation",
}

In [ ]:
NMF_interpretated_topics = retain_interpretated_topics(NMF_relevant_topics, NMF_topic_interpretation)
NMF_interpretated_topics

In [ ]:
df_corpus['labels'] = pd.Series(df_corpus.index).apply(
    lambda i: ';'.join(((df_corpus.at[i, 'labels'],) if df_corpus.at[i, 'labels'] else ()) + (() if i not in NMF_interpretated_topics else NMF_interpretated_topics[i]))
)

#### Export result

In [ ]:
df_corpus['labels'] = df_corpus['labels'].apply(lambda ls: ';'.join(list(dict.fromkeys(ls.split(';')))))

In [ ]:
df_corpus.head(2)

In [ ]:
df_corpus.to_excel(os.path.join(path_to_data, 'data_labels.xlsx'), index = False)

<a id="bottom"></a>

[Table of content](#TOC)